In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [4]:
learning_rate = 0.00000005
meta_step_size = 0.25 #0.25

inner_batch_size = 8
eval_batch_size = 8

meta_iters = 1000 # 2000
eval_iters = 5 # repetitions
inner_iters = 3 # repetitions

eval_interval = 1 #meta_iter % eval_interval == 0이면 eval. 1이면 매번 한다는 뜻.
train_shots = 431
f_shots = 5 # eval
classes = 5
split = 5

In [5]:
import cv2
class Dataset:
    # This class will facilitate the creation of a few-shot dataset
    # from the Omniglot dataset that can be sampled from quickly while also
    # allowing to create new labels at the same time.
    def __init__(self, mode):
        # Download the tfrecord files containing the omniglot data and convert to a
        # dataset.
        self.mode = mode
        self.labels = range(11)

                
    def read_img(self, real_label, shots):
        ds = []
        path = os.path.join(os.getcwd(), 'image10', self.mode, str(real_label))
        files = os.listdir(path)
        sp = random.sample(files, k=shots)
        for s in sp:
            image = cv2.imread(path + "/" + s, cv2.IMREAD_COLOR)  # uint8 image
            norm_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            #image = np.asarray(image).astype('float32')
            #image /= 255.0
            ds.append(norm_image)
        return ds
    
    def get_mini_dataset(
        self, batch_size, repetitions, shots, num_classes, split=False
    ):
        temp_labels = np.zeros(shape=(num_classes * shots))
        temp_images = np.zeros(shape=(num_classes * shots, 331, 331, 3))
        if split:
            test_labels = np.zeros(shape=(num_classes * split))
            test_images = np.zeros(shape=(num_classes * split, 331, 331, 3))

        # Get a random subset of labels from the entire label set.
        label_subset = random.sample(self.labels, k=num_classes)
        for class_idx, class_obj in enumerate(label_subset):
            # Use enumerated index value as a temporary label for mini-batch in
            # few shot learning.
            temp_labels[class_idx * shots : (class_idx + 1) * shots] = class_idx
            # If creating a split dataset for testing, select an extra sample from each
            # label to create the test dataset.
            if split:
                test_labels[class_idx * split : (class_idx + 1) * split] = class_idx
                images_to_split = self.read_img(label_subset[class_idx], split+shots)
                test_images[
                    class_idx * split : (class_idx + 1) * split
                ] = images_to_split[:split]
                temp_images[
                    class_idx * shots : (class_idx + 1) * shots
                ] = images_to_split[split:]
            else:
                # For each index in the randomly selected label_subset, sample the
                # necessary number of images.
                temp_images[
                    class_idx * shots : (class_idx + 1) * shots
                ] = self.read_img(label_subset[class_idx], shots)

        dataset = tf.data.Dataset.from_tensor_slices(
            (temp_images.astype(np.float32), temp_labels.astype(np.int32))
        )
        dataset = dataset.shuffle(shots).batch(batch_size).repeat(repetitions)
        if split:
            return dataset, test_images.astype(np.float32), test_labels.astype(np.int32)
        return dataset


import urllib3

urllib3.disable_warnings()  # Disable SSL warnings that may happen during download.
train_dataset = Dataset(mode='train')
test_dataset = Dataset(mode='val')

In [6]:
img_rows, img_cols, img_channel = 331, 331, 3

base_model = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

In [7]:
add_model = tf.keras.Sequential()
add_model.add(tf.keras.layers.GlobalAveragePooling2D())
add_model.add(tf.keras.layers.BatchNormalization())
add_model.add(tf.keras.layers.Dropout(rate = 0.2))
add_model.add(tf.keras.layers.Dense(units=classes, activation=tf.nn.softmax))

# Compile
model = tf.keras.Model(inputs=base_model.input, outputs=add_model(base_model.output))
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [8]:
"""
add_model = tf.keras.Sequential()
add_model.add(tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(tf.keras.layers.Dense(units=classes, activation=tf.nn.softmax))

model = tf.keras.Model(inputs=base_model.input, outputs=add_model(base_model.output))
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#model.summary()
"""

"\nadd_model = tf.keras.Sequential()\nadd_model.add(tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:]))\nadd_model.add(tf.keras.layers.Dense(units=classes, activation=tf.nn.softmax))\n\nmodel = tf.keras.Model(inputs=base_model.input, outputs=add_model(base_model.output))\noptimizer = tf.keras.optimizers.Adam(lr=learning_rate)\nmodel.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])\n#model.summary()\n"

In [ ]:
training = []
testing = []
for meta_iter in range(meta_iters):
    frac_done = meta_iter / meta_iters
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    # Temporarily save the weights from the model.
    old_vars = model.get_weights()
    # Get a sample from the full dataset.
    mini_dataset = train_dataset.get_mini_dataset(
        inner_batch_size, inner_iters, train_shots, classes
    )
    for images, labels in mini_dataset:
        with tf.GradientTape() as tape:
            preds = model(images)
            loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    new_vars = model.get_weights()
    # Perform SGD for the meta step.
    for var in range(len(new_vars)):
        new_vars[var] = old_vars[var] + (
            (new_vars[var] - old_vars[var]) * cur_meta_step_size
        )
    # After the meta-learning step, reload the newly-trained weights into the model.
    model.set_weights(new_vars)
    # Evaluation loop
    if meta_iter % eval_interval == 0:
        accuracies = []
        for dataset in (train_dataset, test_dataset):
            # Sample a mini dataset from the full dataset.
            train_set, test_images, test_labels = dataset.get_mini_dataset(
                eval_batch_size, eval_iters, f_shots, classes, split=split
            )
            old_vars = model.get_weights()
            # Train on the samples and get the resulting accuracies.
            for images, labels in train_set:
                with tf.GradientTape() as tape:
                    preds = model(images)
                    loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
                grads = tape.gradient(loss, model.trainable_weights)
                optimizer.apply_gradients(zip(grads, model.trainable_weights))
            test_preds = model.predict(test_images)
            print(test_preds)
            test_preds = tf.argmax(test_preds, axis=1).numpy()
            num_correct = (test_preds == test_labels).sum()
            # Reset the weights after getting the evaluation accuracies.
            model.set_weights(old_vars)
            accuracies.append(num_correct / (classes*split))
        training.append(accuracies[0])
        testing.append(accuracies[1])
        #if meta_iter % 100 == 0:
        print(
            "batch %d: train=%f test=%f" % (meta_iter, accuracies[0], accuracies[1])
        )

[[0.21064724 0.23582523 0.16220035 0.15940759 0.23191965]
 [0.25642228 0.18759416 0.15236065 0.20442922 0.19919373]
 [0.18686986 0.18140589 0.16120447 0.20861453 0.26190528]
 [0.15309963 0.23571019 0.17020483 0.18606022 0.2549251 ]
 [0.19646962 0.18504204 0.14982074 0.2007118  0.26795578]
 [0.16367541 0.18459184 0.14683619 0.23803167 0.26686484]
 [0.19469473 0.16716234 0.16424571 0.21327046 0.26062676]
 [0.19991755 0.21097144 0.18341428 0.16697222 0.23872453]
 [0.18877698 0.19280078 0.17581323 0.18913619 0.2534728 ]
 [0.19642161 0.18101205 0.17223394 0.19521149 0.25512096]
 [0.18645337 0.17831554 0.16866764 0.19929986 0.2672636 ]
 [0.16424854 0.22322156 0.15083541 0.20530081 0.25639367]
 [0.17890139 0.21621539 0.13759954 0.22683437 0.24044928]
 [0.20235664 0.19952644 0.15954229 0.18436348 0.25421116]
 [0.17615259 0.20028299 0.17676887 0.1978329  0.24896261]
 [0.18867137 0.1725664  0.1678844  0.21477404 0.2561038 ]
 [0.179949   0.20176066 0.14741081 0.21753822 0.2533413 ]
 [0.20374472 0

[[0.19453357 0.17083351 0.1576772  0.22086155 0.2560942 ]
 [0.1929968  0.20132554 0.15970889 0.18991457 0.25605422]
 [0.19157638 0.18160747 0.16102193 0.20750469 0.25828952]
 [0.19386545 0.18823487 0.17385072 0.19818574 0.24586318]
 [0.19606854 0.18240532 0.15775785 0.21106988 0.25269842]
 [0.19818969 0.21015337 0.16304028 0.18581305 0.24280357]
 [0.20144023 0.2024001  0.15396272 0.18965128 0.25254565]
 [0.2051484  0.17753132 0.16450247 0.20875372 0.2440641 ]
 [0.20506302 0.20239855 0.1553666  0.19222556 0.24494621]
 [0.17109326 0.18965815 0.17766823 0.20315424 0.25842613]
 [0.19463037 0.18244939 0.1735352  0.21059379 0.23879123]
 [0.1877858  0.21909675 0.16219057 0.17835048 0.2525764 ]
 [0.22115651 0.16572253 0.16128749 0.22678822 0.22504517]
 [0.18253833 0.18737891 0.17109984 0.2039516  0.25503138]
 [0.19862212 0.21544819 0.14698532 0.18005435 0.25889003]
 [0.15775606 0.23764087 0.1553754  0.19121401 0.25801364]
 [0.18537416 0.18216659 0.17297624 0.19903228 0.26045075]
 [0.1643087  0

[[0.1989541  0.1942563  0.17597307 0.18914995 0.2416666 ]
 [0.19203557 0.23847991 0.18481147 0.15571581 0.22895725]
 [0.16959532 0.21739174 0.17625923 0.18931279 0.24744087]
 [0.18501735 0.20771465 0.16990523 0.19483931 0.24252337]
 [0.24220675 0.20009774 0.1569356  0.17890245 0.22185743]
 [0.17231444 0.21972813 0.18297483 0.17779651 0.24718612]
 [0.20060576 0.19469063 0.1827902  0.18600912 0.2359043 ]
 [0.18938628 0.19211672 0.17277448 0.19305946 0.25266305]
 [0.20131011 0.1778755  0.15549028 0.22238258 0.24294156]
 [0.19240946 0.19597578 0.17195943 0.20072672 0.23892865]
 [0.19692652 0.17850737 0.17206553 0.21331482 0.2391857 ]
 [0.18229914 0.20376773 0.18524331 0.1941998  0.23449   ]
 [0.19967479 0.17230536 0.16520245 0.21681924 0.24599816]
 [0.19393508 0.19371888 0.1717103  0.19164519 0.24899055]
 [0.17861813 0.19916444 0.16578838 0.23104171 0.22538733]
 [0.19278675 0.17293483 0.17429014 0.21609041 0.24389793]
 [0.1810533  0.17470051 0.18532027 0.21178812 0.24713774]
 [0.18892941 0

[[0.19346054 0.19457163 0.1781574  0.19815196 0.23565851]
 [0.21630691 0.18224488 0.16292381 0.21000262 0.22852181]
 [0.197545   0.17183782 0.17097135 0.22297533 0.23667045]
 [0.18924136 0.17748104 0.18245131 0.21350963 0.23731667]
 [0.18582775 0.17790958 0.16675444 0.22382772 0.24568053]
 [0.17139748 0.19048966 0.17061064 0.2182592  0.24924305]
 [0.19859286 0.17677695 0.17614031 0.21606423 0.23242563]
 [0.21993291 0.18040955 0.16868363 0.21397422 0.21699972]
 [0.19997892 0.17346154 0.17169431 0.21402532 0.2408399 ]
 [0.18970701 0.17637016 0.18386346 0.1984891  0.25157025]
 [0.18227918 0.2134148  0.17091468 0.19676144 0.23662986]
 [0.18409641 0.19019066 0.19409844 0.21476299 0.21685147]
 [0.20188491 0.19663082 0.17483182 0.17609984 0.2505526 ]
 [0.1851354  0.20010339 0.17555991 0.2322901  0.2069112 ]
 [0.21872185 0.17674306 0.1635723  0.23236857 0.20859414]
 [0.19520037 0.21044295 0.13772047 0.24743755 0.20919867]
 [0.18154441 0.17814572 0.182795   0.21501891 0.242496  ]
 [0.17462872 0

[[0.19922578 0.1901929  0.18716997 0.19513974 0.22827168]
 [0.19277881 0.2006626  0.1539339  0.23154278 0.22108191]
 [0.16766071 0.21275887 0.19254287 0.19026011 0.2367775 ]
 [0.18032616 0.2240132  0.19363508 0.18328032 0.21874525]
 [0.1937929  0.20431443 0.19588308 0.1886043  0.2174053 ]
 [0.1917165  0.20062271 0.17651019 0.19119658 0.23995407]
 [0.20000392 0.18500277 0.17578673 0.20373754 0.23546906]
 [0.19067587 0.21684475 0.17730233 0.18012851 0.23504849]
 [0.19967763 0.18729605 0.17773966 0.20450118 0.2307855 ]
 [0.19339588 0.20078814 0.17752114 0.19493183 0.23336302]
 [0.18974212 0.18249333 0.19523463 0.19624189 0.23628804]
 [0.17545122 0.19095619 0.17672123 0.22715463 0.22971678]
 [0.1948948  0.1840471  0.17594121 0.20836727 0.23674962]
 [0.19478652 0.18979879 0.18653348 0.2040935  0.22478767]
 [0.18456146 0.18867153 0.18701604 0.20753297 0.23221806]
 [0.18746169 0.20329294 0.17453422 0.19949289 0.2352183 ]
 [0.17366695 0.19884287 0.16795948 0.21392988 0.24560082]
 [0.20854852 0

...

[[0.2040422  0.19902425 0.19743411 0.19569266 0.20380676]
 [0.20493844 0.19181344 0.19441204 0.20703886 0.20179726]
 [0.20429057 0.19857723 0.19680637 0.20393798 0.19638784]
 [0.19510773 0.2275176  0.19968621 0.18398674 0.19370179]
 [0.21147415 0.19869705 0.19603175 0.19772339 0.19607362]
 [0.22157982 0.18832304 0.18059115 0.20306653 0.20643947]
 [0.1848514  0.19783595 0.20955546 0.19767335 0.2100839 ]
 [0.20722108 0.18719716 0.19591935 0.20755701 0.20210537]
 [0.19881657 0.19320689 0.2028435  0.20570086 0.1994322 ]
 [0.19289772 0.2054388  0.20475255 0.19067442 0.20623653]
 [0.19778304 0.1946445  0.20093119 0.2063256  0.20031565]
 [0.18972202 0.17611963 0.2224141  0.21332152 0.19842276]
 [0.19743717 0.19514082 0.20344798 0.2078377  0.19613631]
 [0.19987112 0.20202176 0.19877009 0.19836949 0.20096757]
 [0.17394885 0.20957063 0.20127428 0.21793506 0.19727112]
 [0.1919417  0.19151454 0.20320544 0.20697576 0.20636259]
 [0.19854349 0.19431435 0.20780618 0.19893026 0.2004057 ]
 [0.19503506 0

[[0.19500247 0.197534   0.20422025 0.20061317 0.20263015]
 [0.19622338 0.19813944 0.20348097 0.20148161 0.20067456]
 [0.19600685 0.19245918 0.20002112 0.21936314 0.19214974]
 [0.20152543 0.19755287 0.20926373 0.20197575 0.18968223]
 [0.20639819 0.20202164 0.19690566 0.1974813  0.19719325]
 [0.20434354 0.20169577 0.1838227  0.21218744 0.19795062]
 [0.19296823 0.19439916 0.19947074 0.20133847 0.21182331]
 [0.19143309 0.2090318  0.20522551 0.19819263 0.19611694]
 [0.20354876 0.19117709 0.20538382 0.19806108 0.20182924]
 [0.20554683 0.20697962 0.19175233 0.1944709  0.20125031]
 [0.20497668 0.18413915 0.206256   0.20082504 0.20380314]
 [0.19768843 0.19965194 0.1968281  0.20759779 0.1982337 ]
 [0.20549676 0.18109953 0.20954551 0.20884933 0.19500887]
 [0.20115711 0.23922238 0.19252256 0.18635103 0.18074697]
 [0.19768499 0.19907826 0.20209336 0.1973805  0.2037629 ]
 [0.17358115 0.21134876 0.20114104 0.21725078 0.19667824]
 [0.2045621  0.19257878 0.2048202  0.19658725 0.20145167]
 [0.20691502 0

[[0.20566322 0.20263402 0.2097286  0.19222927 0.18974492]
 [0.20302401 0.1998639  0.19247715 0.21608958 0.1885454 ]
 [0.20892638 0.20451547 0.18428878 0.20504643 0.19722296]
 [0.19307745 0.18381728 0.19793639 0.22350006 0.20166889]
 [0.20602731 0.19128051 0.19217771 0.20919387 0.20132057]
 [0.19124243 0.1925976  0.19116157 0.22492957 0.20006879]
 [0.18810907 0.21332882 0.20429301 0.19973521 0.1945339 ]
 [0.2027742  0.19773944 0.20317061 0.19866854 0.19764726]
 [0.19937895 0.19588363 0.20657025 0.19519734 0.20296977]
 [0.19927445 0.19484173 0.19787537 0.21008028 0.19792815]
 [0.1978049  0.21446349 0.19263482 0.19838803 0.1967088 ]
 [0.2046146  0.18959312 0.19638164 0.20551181 0.20389886]
 [0.2117727  0.20513049 0.19529396 0.19315748 0.19464542]
 [0.20374739 0.19298775 0.19486706 0.20687942 0.20151839]
 [0.19228233 0.1903171  0.2046632  0.21043812 0.20229928]
 [0.20529486 0.20602298 0.1859389  0.19821037 0.2045329 ]
 [0.18799618 0.19485845 0.22335523 0.19185178 0.2019383 ]
 [0.17822428 0

[[0.19747926 0.2024142  0.1739167  0.22616065 0.20002918]
 [0.2008199  0.18549214 0.20254327 0.19732988 0.2138148 ]
 [0.17361043 0.18996681 0.22502483 0.20545822 0.20593971]
 [0.18578486 0.2018648  0.21192728 0.19694813 0.20347494]
 [0.19566606 0.19869319 0.20822234 0.2011655  0.19625293]
 [0.21439303 0.19572692 0.20087786 0.2025745  0.18642767]
 [0.21681951 0.19332626 0.19204812 0.19901928 0.19878687]
 [0.19512196 0.19365337 0.21718141 0.19927798 0.19476523]
 [0.19477448 0.19250934 0.21793805 0.18716745 0.20761064]
 [0.18945344 0.20194271 0.19927989 0.20329088 0.20603308]
 [0.20410787 0.20145732 0.2000713  0.19772248 0.19664097]
 [0.20507854 0.20645614 0.19336362 0.19626908 0.19883259]
 [0.20156875 0.18511347 0.20757039 0.19885272 0.2068947 ]
 [0.20602076 0.19333903 0.20532319 0.19422491 0.20109212]
 [0.1929991  0.1939238  0.21090725 0.19381772 0.20835216]
 [0.2014373  0.20709766 0.18137072 0.20496279 0.20513152]
 [0.17107394 0.20300369 0.22039163 0.19773068 0.2078001 ]
 [0.1925459  0

[[0.2005636  0.20385931 0.20952049 0.18852882 0.19752774]
 [0.20528209 0.21064164 0.19599202 0.18921071 0.19887352]
 [0.1864738  0.18146731 0.21153335 0.1977168  0.22280869]
 [0.20672494 0.19351722 0.21595962 0.18991828 0.19387992]
 [0.20943214 0.2183694  0.18749501 0.18178865 0.20291482]
 [0.20265988 0.19394675 0.1983272  0.20757145 0.19749475]
 [0.20132238 0.20455427 0.18928422 0.20207179 0.20276734]
 [0.18995848 0.19903883 0.19846061 0.20037559 0.21216646]
 [0.1969351  0.20244896 0.19761913 0.209085   0.1939118 ]
 [0.19684212 0.21490173 0.1928035  0.19855121 0.19690146]
 [0.20429438 0.1868912  0.19005    0.19687594 0.22188847]
 [0.17889492 0.19628106 0.21056448 0.22099485 0.1932647 ]
 [0.19692837 0.19610465 0.19450079 0.2079712  0.20449495]
 [0.18641922 0.21395469 0.20051324 0.19822156 0.20089127]
 [0.20306146 0.19997719 0.19449751 0.21374373 0.18872006]
 [0.19008547 0.18682405 0.20071067 0.21792084 0.20445894]
 [0.19011281 0.20050608 0.20522746 0.20366438 0.20048925]
 [0.19110912 0

In [ ]:
# First, some preprocessing to smooth the training and testing arrays for display.
window_length = 100
train_s = np.r_[
    training[window_length - 1 : 0 : -1], training, training[-1:-window_length:-1]
]
test_s = np.r_[
    testing[window_length - 1 : 0 : -1], testing, testing[-1:-window_length:-1]
]
w = np.hamming(window_length)
train_y = np.convolve(w / w.sum(), train_s, mode="valid")
test_y = np.convolve(w / w.sum(), test_s, mode="valid")

# Display the training accuracies.
x = np.arange(0, len(test_y), 1)
plt.plot(x, test_y, x, train_y)
plt.legend(["test", "train"])
plt.grid()
plt.show()

In [ ]:
train_set, test_images, test_labels = dataset.get_mini_dataset(
    eval_batch_size, eval_iters, train_shots, eval_shots, classes, split=True
)
for images, labels in train_set:
    with tf.GradientTape() as tape:
        preds = model(images)
        loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
test_preds = model.predict(test_images)
test_preds = tf.argmax(test_preds).numpy()
num_correct = (test_preds == test_labels).sum()
print(num_correct / classes)